# Cross-Validation

При перекрестной проверке мы запускаем наш процесс моделирования на разных подмножествах данных, чтобы получить несколько показателей качества модели

#### Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("../datasets/melb_data.csv")

# Нужные подмножества характеристик
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Целевые значения
y = data.Price

#### Перекрестная проверка
Затем определяем конвейер, который использует импутер для заполнения пропущенных значений и модель случайного леса для прогнозирования. Хотя перекрестную проверку и можно выполнить без конвейеров, это довольно сложно. Использование конвейера сделает код удивительно простым.

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


my_pipeline = Pipeline(
    steps=[
        ("preprocessor", SimpleImputer()),
        ("model", RandomForestRegressor(
            n_estimators=50,
            random_state=42,
        ))
    ]
)

Получаем баллы перекрестной проверки с помощью функции cross_val_score() из scikit-learn. Мы устанавливаем количество сгибов с помощью параметра cv

In [3]:
from sklearn.model_selection import cross_val_score

# Умножьте на -1, поскольку sklearn вычисляет *отрицательный* MAE
scores = -1 * cross_val_score(
    my_pipeline, X, y,
    cv=5,
    scoring="neg_mean_absolute_error",
)

print("MAE scores:\n", scores)

MAE scores:
 [297064.72238276 300239.56062646 287168.95506934 237234.84661343
 260589.85501224]


Берем среднее значение по всем экспериментам

In [4]:
print("Average MAE score (across experiments):")
print(scores.mean())

Average MAE score (across experiments):
276459.5879408453
